# Stock Analysis Project
## Historical Price Analysis, Returns, and Risk Metrics

This notebook provides a comprehensive analysis of historical stock price data, computing key financial metrics including returns, volatility, Sharpe ratios, and drawdowns.


## 1. Setup and Data Acquisition

First, we'll import necessary libraries and fetch historical price data for multiple stocks.


In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Add src directory to path - handle both notebook and script execution
if os.path.basename(os.getcwd()) == 'notebooks':
    # We're in the notebooks directory
    project_root = os.path.dirname(os.getcwd())
else:
    # We're in the project root
    project_root = os.getcwd()

sys.path.insert(0, os.path.join(project_root, 'src'))

from data_loader import fetch_prices, save_raw_data
from indicators import prepare_prices, max_drawdown
from analysis import (
    compute_daily_returns,
    compute_cumulative_returns,
    annual_metrics,
    rolling_volatility_annualized,
    prepare_returns,
    compute_summary_stats
)
from visualizations import (
    plot_equity_curves,
    plot_performance_table,
    plot_rolling_volatility,
    plot_max_drawdown,
    plot_sharpe_ratios,
    create_summary_charts
)

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")


### 1.1 Fetch Historical Price Data

We'll analyze 5 major tech stocks: Apple (AAPL), Microsoft (MSFT), Google (GOOGL), Amazon (AMZN), and Tesla (TSLA).


In [ ]:
# Define tickers and date range
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']
start_date = '2018-01-01'
end_date = None  # None means up to today

print(f"Fetching data for {len(tickers)} stocks: {', '.join(tickers)}")
print(f"Date range: {start_date} to {'Today' if end_date is None else end_date}")

# Fetch price data
prices_raw = fetch_prices(tickers, start=start_date, end=end_date)

print(f"\nRaw data shape: {prices_raw.shape}")
print(f"Date range: {prices_raw.index.min()} to {prices_raw.index.max()}")
print(f"\nFirst 5 rows:")
print(prices_raw.head())
print(f"\nLast 5 rows:")
print(prices_raw.tail())


### 1.2 Save Raw Data

Let's save the raw data for future reference.


In [ ]:
# Save raw data
data_dir = os.path.join(project_root, 'data')
save_raw_data(prices_raw, output_dir=data_dir, filename='raw_prices.csv')


## 2. Data Cleaning and Preparation

We'll clean the data by handling missing values and ensuring proper alignment.


In [ ]:
# Clean and prepare prices
prices_clean = prepare_prices(prices_raw)

print(f"Clean prices shape: {prices_clean.shape}")
print(f"\nMissing values per column:")
print(prices_clean.isna().sum())
print(f"\nPercentage of missing values:")
print((prices_clean.isna().sum() / len(prices_clean) * 100).round(2))

# Save clean prices
prices_clean.to_csv(os.path.join(data_dir, 'clean_prices.csv'))
print(f"\nClean prices saved to data/clean_prices.csv")


## 3. Core Analytics: Returns and Cumulative Returns

We'll compute daily returns and cumulative returns using Pandas.


In [ ]:
# Compute returns
daily_returns = compute_daily_returns(prices_clean)
cum_returns = compute_cumulative_returns(daily_returns)

print("Daily Returns Statistics:")
print(daily_returns.describe().round(4))
print(f"\nDaily Returns (last 5 days):")
print(daily_returns.tail())
print(f"\nCumulative Returns (last 5 days):")
print(cum_returns.tail())


### 3.1 Validation: Cumulative Returns

Let's validate that cumulative returns are computed correctly using NumPy.


In [ ]:
# Validation: Compute cumulative return using np.prod
for ticker in tickers:
    if ticker in daily_returns.columns:
        arr = daily_returns[ticker].values
        arr = arr[~np.isnan(arr)]
        cum_return_np = np.prod(1 + arr) - 1
        cum_return_pandas = cum_returns[ticker].iloc[-1]
        
        print(f"{ticker}:")
        print(f"  NumPy cumulative return: {cum_return_np:.4f}")
        print(f"  Pandas cumulative return: {cum_return_pandas:.4f}")
        print(f"  Difference: {abs(cum_return_np - cum_return_pandas):.8f}")
        print()


## 4. Annualized Metrics (NumPy Implementation)

We'll compute annualized return, annualized volatility, and Sharpe ratio using NumPy.


In [ ]:
# Compute annualized metrics
risk_free_rate = 0.0  # Can be updated to actual risk-free rate
stats = annual_metrics(daily_returns, rf=risk_free_rate)

# Add max drawdown
max_dd = max_drawdown(prices_clean)
stats['max_drawdown'] = max_dd

# Format for display
stats_display = stats.copy()
for col in ['annual_return', 'annual_vol', 'total_return', 'max_drawdown']:
    if col in stats_display.columns:
        stats_display[col] = stats_display[col].apply(lambda x: f"{x:.2%}")

print("Summary Statistics:")
print("=" * 80)
print(stats_display)
print("\n" + "=" * 80)
print("\nDetailed Statistics (numeric):")
print(stats)


### 4.1 Performance Ranking

Let's identify the top performers across different metrics.


In [ ]:
print("TOP PERFORMER (Highest Annual Return):")
top_return = stats['annual_return'].idxmax()
print(f"  {top_return}: {stats.loc[top_return, 'annual_return']:.2%}\n")

print("LOWEST VOLATILITY (Most Stable):")
lowest_vol = stats['annual_vol'].idxmin()
print(f"  {lowest_vol}: {stats.loc[lowest_vol, 'annual_vol']:.2%}\n")

print("BEST SHARPE RATIO (Best Risk-Adjusted Return):")
best_sharpe = stats['sharpe'].idxmax()
print(f"  {best_sharpe}: {stats.loc[best_sharpe, 'sharpe']:.4f}\n")

print("WORST MAX DRAWDOWN (Largest Decline):")
worst_dd = stats['max_drawdown'].idxmin()
print(f"  {worst_dd}: {stats.loc[worst_dd, 'max_drawdown']:.2%}\n")


## 5. Rolling Statistics

We'll compute rolling mean and rolling volatility over a 20-day window.


In [ ]:
# Compute rolling statistics
window = 20
rolling_mean_returns = daily_returns.rolling(window=window).mean()
rolling_vol = rolling_volatility_annualized(daily_returns, window=window)

print(f"Rolling Mean Returns ({window}-day window) - Last 5 days:")
print(rolling_mean_returns.tail())
print(f"\nRolling Volatility Annualized ({window}-day window) - Last 5 days:")
print(rolling_vol.tail())


## 6. Visualizations

Let's create comprehensive charts to visualize the analysis results.


In [ ]:
# Create output directory for charts
slides_dir = os.path.join(project_root, 'slides')
os.makedirs(slides_dir, exist_ok=True)

# Generate all charts
create_summary_charts(prices_clean, daily_returns, cum_returns, rolling_vol, stats, output_dir=slides_dir)


### 6.1 Equity Curves

The equity curves show cumulative returns over time, allowing us to compare performance across stocks.


In [ ]:
plot_equity_curves(cum_returns, title="Cumulative Returns (Equity Curves)")
plt.show()


### 6.2 Performance Metrics

Bar charts comparing annualized returns and volatility across stocks.


In [ ]:
plot_performance_table(stats, title="Performance Metrics Comparison")
plt.show()


### 6.3 Rolling Volatility

Time-series of rolling volatility with shaded high-volatility periods.


In [ ]:
plot_rolling_volatility(rolling_vol, title="Rolling Volatility (20-day window, Annualized)")
plt.show()


### 6.4 Drawdown Analysis

Visualization of drawdowns from peak values.


In [ ]:
plot_max_drawdown(prices_clean, title="Maximum Drawdown Analysis")
plt.show()


### 6.5 Sharpe Ratio Comparison

Comparison of risk-adjusted returns (Sharpe ratios) across stocks.


In [ ]:
plot_sharpe_ratios(stats, title="Sharpe Ratio Comparison (Risk-Adjusted Returns)")
plt.show()


## 7. Findings and Interpretation

### Key Findings

Based on our analysis of 5 major tech stocks from 2018 to present:


In [ ]:
# Generate summary findings
print("=" * 80)
print("KEY FINDINGS")
print("=" * 80)
print(f"\nAnalysis Period: {prices_clean.index.min().strftime('%Y-%m-%d')} to {prices_clean.index.max().strftime('%Y-%m-%d')}")
print(f"Number of trading days: {len(prices_clean)}")
print(f"\nStocks Analyzed: {', '.join(tickers)}")

print("\n" + "-" * 80)
print("PERFORMANCE RANKINGS")
print("-" * 80)

# Sort by annual return
stats_sorted_return = stats.sort_values('annual_return', ascending=False)
print("\n1. ANNUALIZED RETURN (Top to Bottom):")
for idx, (ticker, row) in enumerate(stats_sorted_return.iterrows(), 1):
    print(f"   {idx}. {ticker}: {row['annual_return']:.2%}")

# Sort by Sharpe ratio
stats_sorted_sharpe = stats.sort_values('sharpe', ascending=False)
print("\n2. SHARPE RATIO (Best Risk-Adjusted Returns):")
for idx, (ticker, row) in enumerate(stats_sorted_sharpe.iterrows(), 1):
    print(f"   {idx}. {ticker}: {row['sharpe']:.4f}")

# Sort by volatility
stats_sorted_vol = stats.sort_values('annual_vol', ascending=True)
print("\n3. VOLATILITY (Lowest to Highest):")
for idx, (ticker, row) in enumerate(stats_sorted_vol.iterrows(), 1):
    print(f"   {idx}. {ticker}: {row['annual_vol']:.2%}")

print("\n" + "=" * 80)
print("METHODOLOGY")
print("=" * 80)
print("""
1. Data Acquisition: Fetched historical adjusted close prices using yfinance
2. Data Cleaning: Forward-filled missing values and aligned trading days
3. Returns Calculation: 
   - Daily returns: r_t = (P_t / P_{t-1}) - 1
   - Cumulative returns: computed as (1 + daily_returns).cumprod() - 1
4. Annualized Metrics (assuming 252 trading days/year):
   - Annualized Return: mean_daily_return * 252
   - Annualized Volatility: std_daily_return * sqrt(252)
   - Sharpe Ratio: (annualized_return - risk_free_rate) / annualized_volatility
5. Rolling Statistics: 20-day rolling window for mean and volatility
6. Risk Metrics: Maximum drawdown from peak values
""")

print("\n" + "=" * 80)
print("SUMMARY TABLE")
print("=" * 80)
print(stats.round(4))


### 7.1 Export Summary Statistics

Save the summary statistics to CSV for external analysis.


In [ ]:
# Save summary statistics
stats.to_csv(os.path.join(data_dir, 'summary_stats.csv'))
print(f"Summary statistics saved to data/summary_stats.csv")
stats.head()
